In [1]:
import json
import itertools
from fractions import Fraction
import numpy as np

In [2]:
%run logic.ipynb


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [3]:
#Example JSON
root_question = {'qid':0,
'question': 'Blood type?',
'answers': ['O+','A+','Other'],
'probability': ['1/3','1/3','1/3']}

follow_up_1 = {'qid':1,
'question': 'Specify...',
'answers': ['B+','AB+','O-','A-','Other'],
'probability': ['1/5','1/5','1/5','1/5','1/5']}

follow_up_2 = {'qid':2,
'question': 'Specify further...',
'answers': ['B-','AB-'],
'probability': ['1/2','1/2']}

paths = [[0,'Other',1], [1,'Other',2]]

json_poll = {
    'roots': [root_question],
    'children': [follow_up_1, follow_up_2],
    'paths':paths,
    'truth':['1/4']
}
json_poll

{'children': [{'answers': ['B+', 'AB+', 'O-', 'A-', 'Other'],
   'probability': ['1/5', '1/5', '1/5', '1/5', '1/5'],
   'qid': 1,
   'question': 'Specify...'},
  {'answers': ['B-', 'AB-'],
   'probability': ['1/2', '1/2'],
   'qid': 2,
   'question': 'Specify further...'}],
 'paths': [[0, 'Other', 1], [1, 'Other', 2]],
 'roots': [{'answers': ['O+', 'A+', 'Other'],
   'probability': ['1/3', '1/3', '1/3'],
   'qid': 0,
   'question': 'Blood type?'}],
 'truth': ['1/4']}

In [4]:
#Parse to proper datatypes
root_questions = json_poll['roots']
follow_ups = json_poll['children']
root_to_truth = {}

#Convert from id to questions
qid_dict = {}

count = 0
for root in root_questions:
    root['probability'] = map(lambda x: Fraction(x), root['probability']) #Convert String to Fraction
    qid_dict[root['qid']]= root
    root_to_truth[root['qid']] = Fraction(json_poll['truth'][count])
    count+=1

    
for followup in follow_ups:
    followup['probability'] = map(lambda x: Fraction(x), followup['probability']) #Convert String to Fraction
    qid_dict[followup['qid']] = followup

In [5]:
#Find question order

subtrees = {}
edges = []
question_paths = {}
answer_prob = {}

for root in root_questions:
    qid = root['qid']
    subtrees[qid] = [root]
    traverse(qid_dict, paths, subtrees[qid], edges, qid)
    decorate(qid_dict, paths, subtrees[qid])
    
    question_paths[qid] = findPath(qid_dict, paths, [], qid)
    answer_prob[qid] = findProbabilities(qid_dict, paths, [], qid)

In [6]:
#Construct transition matrix
transition_matrix_subtree = {}
epsilons = {}
for tree in answer_prob.keys():
    coinflip = root_to_truth[tree]
    path_prob = {}

    for path in answer_prob[tree]:
        node_prob = 1
        path_string = weightedPathToKey(path)
        for alt, prob in path:
             node_prob*=prob
        path_prob[path_string] = node_prob
    
    matrix = {}
    chosen_epsilon = 0
    for _from in path_prob.keys():
        column_keys = []
        for _to in path_prob.keys():
            key = (_from, _to)
            column_keys.append(key)
            # Higher likelihood to stay on same value
            if _from == _to:
                matrix[key] = (1-coinflip)*path_prob[_to] + coinflip #Random + truth
            else:
                matrix[key]= (1-coinflip)*path_prob[_to] #Only random
            
        cases = []
        for key in column_keys:
            cases.append(matrix[key])
        
        assert sum(cases)==1
        
        candidate_epsilon=  max(np.divide(min(cases),max(cases)), np.divide(max(cases),min(cases)))
        if candidate_epsilon>chosen_epsilon:
            chosen_epsilon=candidate_epsilon
    
    print chosen_epsilon
        
    epsilons[tree] = chosen_epsilon
    transition_matrix_subtree[tree] = matrix
   

20


In [7]:
total_epsilon = sum([epsilons[x] for x in epsilons.keys()])

In [8]:
transition_matrix_subtree[0]

{('A+', 'A+'): Fraction(1, 2),
 ('A+', 'O+'): Fraction(1, 4),
 ('A+', 'OtherA-'): Fraction(1, 20),
 ('A+', 'OtherAB+'): Fraction(1, 20),
 ('A+', 'OtherB+'): Fraction(1, 20),
 ('A+', 'OtherO-'): Fraction(1, 20),
 ('A+', 'OtherOtherAB-'): Fraction(1, 40),
 ('A+', 'OtherOtherB-'): Fraction(1, 40),
 ('O+', 'A+'): Fraction(1, 4),
 ('O+', 'O+'): Fraction(1, 2),
 ('O+', 'OtherA-'): Fraction(1, 20),
 ('O+', 'OtherAB+'): Fraction(1, 20),
 ('O+', 'OtherB+'): Fraction(1, 20),
 ('O+', 'OtherO-'): Fraction(1, 20),
 ('O+', 'OtherOtherAB-'): Fraction(1, 40),
 ('O+', 'OtherOtherB-'): Fraction(1, 40),
 ('OtherA-', 'A+'): Fraction(1, 4),
 ('OtherA-', 'O+'): Fraction(1, 4),
 ('OtherA-', 'OtherA-'): Fraction(3, 10),
 ('OtherA-', 'OtherAB+'): Fraction(1, 20),
 ('OtherA-', 'OtherB+'): Fraction(1, 20),
 ('OtherA-', 'OtherO-'): Fraction(1, 20),
 ('OtherA-', 'OtherOtherAB-'): Fraction(1, 40),
 ('OtherA-', 'OtherOtherB-'): Fraction(1, 40),
 ('OtherAB+', 'A+'): Fraction(1, 4),
 ('OtherAB+', 'O+'): Fraction(1, 4)

In [9]:
question_paths[0]

[['O+'],
 ['A+'],
 ['Other', 'B+'],
 ['Other', 'AB+'],
 ['Other', 'O-'],
 ['Other', 'A-'],
 ['Other', 'Other', 'B-'],
 ['Other', 'Other', 'AB-']]

In [10]:
subtrees[0]

[{'answers': ['O+', 'A+', 'Other'],
  'hasFollowUp': 'True',
  'probability': [Fraction(1, 3), Fraction(1, 3), Fraction(1, 3)],
  'qid': 0,
  'question': 'Blood type?'},
 {'answers': ['B+', 'AB+', 'O-', 'A-', 'Other'],
  'hasFollowUp': 'True',
  'probability': [Fraction(1, 5),
   Fraction(1, 5),
   Fraction(1, 5),
   Fraction(1, 5),
   Fraction(1, 5)],
  'qid': 1,
  'question': 'Specify...'},
 {'answers': ['B-', 'AB-'],
  'hasFollowUp': 'False',
  'probability': [Fraction(1, 2), Fraction(1, 2)],
  'qid': 2,
  'question': 'Specify further...'}]